<a href="https://colab.research.google.com/github/raulc27/python_testes/blob/master/HipoteseInferencia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Livro Data Science do Zero

In [14]:
from typing import Tuple
import math

In [17]:
def normal_approximation_to_binomial(n: int, p: float) -> Tuple[float, float]:
  """ Retorna mu e sigma correspondentes Binomial(n,p) """
  mu = p * n
  sigma = math.sqrt(p * (1-p) * n)
  return mu, sigma

In [3]:
def normal_cdf(x: float, mu: float=0, sigma: float=1) -> float:
  return (1 + math.erf((x-mu)/math.sqrt(2)/sigma))/2

In [4]:
normal_probability_below = normal_cdf

In [6]:
def normal_probability_above(lo: float,
                             mu: float=0,
                             sigma: float=1) -> float:
                             """ A probabilidade de que um N (mu, sigma) seja maior do que lo """
                             return 1 - normal_cdf(lo, mu, sigma)

In [7]:
# está entre se é menor do que hi, mas n é menor do que lo
def normal_probability_between(
    lo: float,
    hi: float,
    mu: float = 0,
    sigma: float = 1) -> float:
    """ A probabilidade de que um N (mu, sigma) esteja entre lo e hi. """
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

In [8]:
# Está fora se não está entre
def normal_probability_outside(lo: float,
                               hi: float,
                               mu: float = 0,
                               sigma: float =1) -> float:
                               """ A probabilidade de que um N (mu, sigma) não esteja entre lo e hi. """
                               return 1 - normal_probability_between(lo, hi, mu, sigma)

In [9]:
def inverse_normal_cdf(p: float,
                       mu: float=0,
                       sigma: float=1,
                       tolerance: float=0.00001) -> float:
                       """ encontrar o inverso aproximado usando a pesquisa binária! """

                       # se não for padrão, compute o padrão e redimensione
                       if mu != 0 or sigma != 1:
                         return mu + sigma * inverse_normal_cdf(p, tolerance=tolerance)

                       low_z = -10.0
                       hi_z = 10.0
                       while hi_z - low_z > tolerance:
                         mid_z = (low_z + hi_z) / 2
                         mid_p = normal_cdf(mid_z)
                         if mid_p < p:
                           low_z = mid_z
                         else:
                           hi_z = mid_z
                       return mid_z

In [10]:
def normal_upper_bound(
    probability: float,
    mu: float = 0,
    sigma: float = 1) -> float:
    """ Retorna o z para o qual P(Z<=z) = probabilidade """
    return inverse_normal_cdf(probability, mu, sigma)

In [11]:
def normal_lower_bound(
    probability: float,
    mu: float = 0,
    sigma: float = 1
) -> float:
  """ Retorna o z para o qual P(Z >= z) = probabilidade """
  return inverse_normal_cdf(1-probability, mu, sigma)

In [15]:
def normal_two_sided_bounds(probability: float,
                            mu: float = 0,
                            sigma: float = 1) -> Tuple[float, float]:
                            """
                            Retorna os limites simétricos (relativos à média)
                            que contém a probabilidade especificada
                            """
                            tail_probability = (1 - probability) / 2

                            # O limite superior deve estar abaixo de tail_probability
                            upper_bound = normal_lower_bound(tail_probability, mu, sigma)

                            # O limite inferior deve estar acima de tail_probability/
                            lower_bound = normal_upper_bound(tail_probability, mu, sigma)

                            return lower_bound, upper_bound 

In [18]:
# Se hipótese de honestidade estiver correta, X estará distribuído aproximadamente em um função Normal
mu_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)

In [19]:
# Teste recusará Ho se X estiver fora dos limites obtidos da seguinte forma:
lower_bound, upper_bound = normal_two_sided_bounds(0.95, mu_0, sigma=0)

In [20]:
# calculando potência do teste
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)

In [21]:
mu_1, sigma_1 = normal_approximation_to_binomial(1000,0.55)

In [22]:
# erro tipo 2 ocorre qdo rejeitamos uma hipótese nula (o que ocorre qdo X ainda está no intervalo original)
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)
power = 1 - type_2_probability

In [23]:
hi = normal_upper_bound(0.95, mu_0, sigma_0)
type_2_probability = normal_probability_below(hi, mu_1, sigma_1)
power = 1 - type_2_probability